In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2

In [4]:
import pandas as pd
from descriptors.dataset import MoleculesDataset, mol_collate_func
from torch.utils.data import DataLoader
from descriptastorus.descriptors.DescriptorGenerator import MakeGenerator
from tqdm import tqdm

In [54]:
chembl = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/chembl_4_smiles.csv',
                    index_col=0)

In [55]:
all_toxs = []

for index, row in tqdm(chembl.iterrows()):
    for i in row['chembl_tox'].split('+'):
        all_toxs.append(i)


0it [00:00, ?it/s]
566it [00:00, 5658.14it/s]
1132it [00:00, 3535.09it/s]
2164it [00:00, 4051.64it/s]


In [38]:
all_tox = list(set(all_toxs))

In [39]:
all_tox.pop(15)

'Safe'

In [53]:
all_tox

['toxic_hepa',
 'toxic_carcinogenic',
 'toxic_gastro',
 'toxic_immune',
 'toxic_neuro',
 'toxic_muscoskelet',
 'toxic_teratogen',
 'toxic_cardio',
 'toxic_dermis',
 'toxic_nephro',
 'toxic_hema',
 'toxic_metabolism',
 'toxic_psyche',
 'toxic_respiratory',
 'toxic_vascular',
 'infections']

In [57]:
chembl['chembl_toxicity_updated'] = 'Safe'

In [74]:
for index, row in tqdm(chembl.iterrows()):
    molecule = row['chembl_id']
    if len(row['chembl_tox'].split('+')) > 1:
        chembl.loc[chembl['chembl_id'] == molecule, 'chembl_toxicity_updated'] = 'multiple'
    else:
        chembl.loc[chembl['chembl_id'] == molecule, 'chembl_toxicity_updated'] = row['chembl_tox']

2164it [00:06, 328.82it/s]


In [76]:
chembl['chembl_toxicity_updated'].value_counts()

Safe                  1894
multiple               118
toxic_psyche            42
toxic_teratogen         30
toxic_cardio            19
toxic_respiratory       16
toxic_hepa              16
toxic_hema               8
toxic_vascular           7
toxic_carcinogenic       5
toxic_neuro              3
toxic_nephro             2
toxic_metabolism         2
toxic_gastro             1
toxic_dermis             1
Name: chembl_toxicity_updated, dtype: int64

In [77]:
chembl

,chembl_id,pubchem_cid,smiles,parent_smiles,chembl_tox,withdrawn,chembl_toxicity_updated
258,CHEMBL1742471,65869,N=C(N)Nc1nc(CSCCN/C=N/S(=O)(=O)c2ccc(Br)cc2)cs1,N=C(N)Nc1nc(CSCCN/C=N/S(=O)(=O)c2ccc(Br)cc2)cs1,Safe,1,Safe
276,CHEMBL666,3415,O=C(O)P(=O)(O)O,O=C(O)P(=O)(O)O,toxic_nephro+toxic_neuro,0,multiple
280,CHEMBL1580,439693,OC[C@H]1O[C@@H](n2cnc3c2N=CNC[C@H]3O)C[C@@H]1O,OC[C@H]1O[C@@H](n2cnc3c2N=CNC[C@H]3O)C[C@@H]1O,Safe,0,Safe
281,CHEMBL606,5282381,CCCCC(C)(O)C/C=C/[C@H]1[C@H](O)CC(=O)[C@@H]1CC...,CCCCC(C)(O)C/C=C/[C@H]1[C@H](O)CC(=O)[C@@H]1CC...,toxic_teratogen,0,toxic_teratogen
303,CHEMBL110,31593,O=C(Cn1ccnc1[N+](=O)[O-])NCc1ccccc1,O=C(Cn1ccnc1[N+](=O)[O-])NCc1ccccc1,Safe,0,Safe
...,...,...,...,...,...,...,...
10666,CHEMBL1089318,135565903,Cc1c(Cl)c(C)[n+]([O-])c(Cl)c1-c1noc(-c2cc(O)c(...,Cc1c(Cl)c(C)[n+]([O-])c(Cl)c1-c1noc(-c2cc(O)c(...,Safe,0,Safe
10667,CHEMBL1200624,9270,C#C[C@]1(OC(C)=O)CC[C@H]2[C@@H]3CCC4=C[C@@H](O...,C#C[C@]1(OC(C)=O)CC[C@H]2[C@@H]3CCC4=C[C@@H](O...,Safe,0,Safe
10672,CHEMBL15891,missing,Cl[C@H]1[C@H](Cl)[C@@H](Cl)[C@@H](Cl)[C@H](Cl)...,Cl[C@H]1[C@H](Cl)[C@@H](Cl)[C@@H](Cl)[C@H](Cl)...,Safe,0,Safe
10677,CHEMBL90,774,NCCc1c[nH]cn1,NCCc1c[nH]cn1,Safe,0,Safe


In [99]:
chembl_plus = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/chembl_drugbank_p4_21_7_+missing_labels.csv', index_col=0)

In [100]:
withdrawn = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/raw/withdrawn.csv')

In [101]:
chembl_plus = chembl_plus.merge(withdrawn[['CHEMBLID', 'Toxicity type']], how='left', left_on='chembl_id', right_on='CHEMBLID')

In [104]:
chembl_plus['Toxicity type'] = chembl_plus['Toxicity type'].fillna('Safe')

In [105]:
chembl_plus['Toxicity type'].value_counts()

Safe                                                  2442
toxic_hepa                                              56
toxic_cardio                                            50
toxic_neuro                                             29
toxic_hema                                              28
toxic_carcinogenic                                      23
toxic_systemic                                          21
toxic_dermis                                            16
toxic_nephro                                            13
toxic_teratogen                                          9
toxic_respiratory                                        8
toxic_opthalmic                                          4
toxic_muscoskelet                                        4
toxic_gastro                                             2
toxic_hepa, toxic_nephro, toxic_dermis, toxic_hema       1
carinogenicity                                           1
toxic_carcinogenic, toxic_nephro                        

In [134]:
rows = []
for index, row in tqdm(chembl_plus.iterrows()):
    toxicities = []
    if len(row['Toxicity type'].split(', ')) > 1:
        for i in row['Toxicity type'].split(', '):
            toxicities.append(i)
    else:
        toxicities.append(row['Toxicity type'])
    
    row1 = row
    row2 = row
    row3 = row
    row4 = row
    new_rows = [row1, row2, row3, row4]
    for i in range(len(toxicities)):
        new_rows[i]['Toxicity type'] = toxicities[i]
        rows.append(new_rows[i])

2722it [00:00, 4918.62it/s]


In [140]:
chembl_plus = pd.DataFrame(rows)

In [141]:
chembl_plus.rename(columns={'Toxicty type': 'withdrawn_toxicity'}, inplace=True)

In [142]:
chembl_plus.drop_duplicates(inplace=True)

In [143]:
chembl_plus.to_csv('/home/dionizije/Documents/drug_attrition_oracle/data/chembl_drugbank_p4_21_7_+missing_labels+withdrawn_tox.csv')